In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

soup = BeautifulSoup(requests.get(f"https://books.toscrape.com/catalogue/category/books/travel_2/index.html").content,"lxml")

section_dicto = {}
for i in range(1,51):
    sections = soup.find('ul',{"class":"nav nav-list"}).find_all("a")[i].get_text().strip()
    section_dicto[sections] = sections.lower()+f"_{i+1}"

input = input("Enter the section : ")

soup = BeautifulSoup(requests.get(f"http://books.toscrape.com/catalogue/category/books/{section_dicto[input.capitalize()]}/index.html").content,"lxml")

rate_dict = {"One":"1 star","Two":"2 stars","Three":"3 stars","Four":"4 stars","Five":"5 stars"}

with open("books.csv","w",newline='',encoding="UTF-8") as fd:

    heads = ["Title","Rate","Price"]
    writer = csv.DictWriter(fd,heads)
    writer.writeheader()

    num_of_pages = soup.find("li",{"class":"current"})
    
    if num_of_pages == None:
        num_of_pages = 1
    else:
        num_of_pages = soup.find("li",{"class":"current"}).get_text().split()[-1]

    for page in range(1,int(num_of_pages)+1):

        if num_of_pages == 1:
            url = f"http://books.toscrape.com/catalogue/category/books/{section_dicto[input.capitalize()]}/index.html"
        else:
            url = f'http://books.toscrape.com/catalogue/category/books/{section_dicto[input.capitalize()]}/page-{str(page)}.html'

        resp = requests.get(url)
        soup = BeautifulSoup(resp.content,"lxml")

        all_articles = soup.find_all("article",{"class":"product_pod"})

        for article in all_articles:
            title = article.find("h3").find("a").attrs["title"]
            price = article.find("p",{"class":"price_color"}).get_text()
            rate = rate_dict[article.find("p",{"class":"star-rating"}).attrs["class"][1]]

            writer.writerow({"Title":title,"Rate":rate,"Price":price})


In [2]:
data = pd.read_csv("books.csv")

data

,Title,Rate,Price
0,Sharp Objects,4 stars,£47.82
1,"In a Dark, Dark Wood",1 star,£19.63
2,The Past Never Ends,4 stars,£56.50
3,A Murder in Time,1 star,£16.64
4,The Murder of Roger Ackroyd (Hercule Poirot #4),4 stars,£44.10
5,The Last Mile (Amos Decker #2),2 stars,£54.21
6,That Darkness (Gardiner and Renner #1),1 star,£13.92
7,Tastes Like Fear (DI Marnie Rome #3),1 star,£10.69
8,A Time of Torment (Charlie Parker #14),5 stars,£48.35
9,A Study in Scarlet (Sherlock Holmes #1),2 stars,£16.73
